#### Main takeaway: Best regime = adjusting $P_0(t)$ for inflation every cycle (relative to $N=0$) in accumulation phase. This way beat inflation thru 2 contributions: market return + actively yourself

#### Formulae:
i) Inflation: $P_0(N)=P_0(0)(1+IR)^N$ for $P_0(N)$ with constant purchasing power 

ii) Regime 1: $P_N(N) = P_N(0)j^N$

iii) Regime 2: $P_N(N) = P_N(0)j^N - \frac{pj(j^N-1)}{i}$

iv) Regime 3: $P_N(N) = [P_0([0:k])](\sum_{k=0}^{N}j^k) = [P_0([0:k])]\frac{j(j^N-1)}{i} $($= P_0\frac{j(j^N-1)}{i}$ if $P_0(t)$ is constant)  

#### Variables:
- $P_N(k)$ = portfolio size after $k$ cycles, $k\in[0,N]$
- $[P_0([0:k])]$ is $1\times k$-size array if $P_0(t)$ is not constant; elementwise multiplication with $\sum_{k=0}^{N}j^k$. This can be used to input inflation-adjusted timeseries $[P_0(t)]$ or otherwise variable timeseries to any one of these functions
- $i$ = $i_{nominal} - i_{inflation}$ = net market interest rate 
- $j$ = $i + 1$
- $p$ = withdrawal rate (per cycle)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Performs inflation adjustment on P0: use to input infl-adj amounts to compound regime
def P0_infl(P0,IR,N):
    return P0*np.array([(1+IR)**yr for yr in range(N+1)])

# Regime 1: ONE-TIME INPUT of P0 and compound with no further input/withdrawal
def PN_cste(P0,i,N):
    j=i+1
    return P0*np.array([j**n for n in range(N+1)])

# Regime 2: WITHDRAWAL w/ no input on PN_0 (=portfolio size at N=0)   
def PN_with(PN_0,i,p,N):
    j=1+i
    if type(PN_0)==np.ndarray:
        return np.array([PN_0[n]*j**n-(p*j*(j**n-1))/i for n in range(N+1)])
    else:
        return np.array([PN_0*j**n-(p*j*(j**n-1))/i for n in range(N+1)]) 
    
# Regime 3: ACCUMULATION of P_0(t) = cste/variable w/o withdrawal
def PN_accum(P0,i,N):
    j=i+1
    output = []
    for n in range(1,N+1):
        jk = [j**k for k in range(1,n+1)]
        output.append(np.dot(P0[:n],jk[::-1]))
    return np.array(output)

# Calculates ROI on k,k+1 cycle:


In [4]:
# Manually inputting inflation vs. nominal interest growth
i = 0.1
j = i+1
N = 10
P0 = 19.5e3
IR = 0.034

P0 = P0_infl(P0,IR,N)
print(P0)
PN_accum(P0,i,N)

[19500.         20163.         20848.542      21557.392428
 22290.34377055 23048.21545875 23831.85478435 24642.13784702
 25479.97053381 26346.28953196 27242.06337605]


array([ 21450.        ,  45774.3       ,  73285.1262    , 104326.7704908 ,
       139278.82568749, 178559.74526086, 222630.76004973, 272000.18768642,
       327228.17404226, 388931.90993165])

In [3]:
# Baking inflation into averaged annual effective interest rate
i = 0.06
j = i+1
N = 20
P0 = 19.5e3
IR = 0

P0 = P0_infl(P0,IR,N)
print(P0)
PN_accum(P0,i,N)

[19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500.
 19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500. 19500.
 19500.]


array([ 20670.        ,  42580.2       ,  65805.012     ,  90423.31272   ,
       116518.7114832 , 144179.83417219, 173500.62422252, 204580.66167587,
       237525.50137643, 272447.03145901, 309463.85334655, 348701.68454735,
       390293.78562019, 434381.4127574 , 481114.29752284, 530651.15537421,
       583160.22469667, 638819.83817847, 697819.02846918, 760358.17017733])

In [7]:
P0=6e3
P0 = P0_infl(P0,IR,N)
PN = PN_accum(P0,i,N)
PN

array([  6600.        ,  14084.4       ,  22549.2696    ,  32100.5447664 ,
        42855.02328846,  54941.46008027,  68501.77232299,  83692.36544198,
       100685.592013  , 119671.35690205])

In [23]:
# Interest return in [k,k+1]
[PN[n+1]-PN[n]-P0[n+1] for n in range(len(PN)-1)]

[2945.600000000006,
 4648.610400000001,
 6521.723353600002,
 8578.576361622418,
 10833.82038089756,
 13303.192766436674,
 16003.595387565369,
 18953.17828250727,
 22171.42924150072,
 25679.269735917085,
 29499.157640158162,
 33655.19722446865,
 38173.256930363816,
 43081.09547627809]

In [32]:
# SWR 4% on various principals
arr = np.arange(100e3,900e3,50e3)
arr,.04*arr

(array([100000., 150000., 200000., 250000., 300000., 350000., 400000.,
        450000., 500000., 550000., 600000., 650000., 700000., 750000.,
        800000., 850000.]),
 array([ 4000.,  6000.,  8000., 10000., 12000., 14000., 16000., 18000.,
        20000., 22000., 24000., 26000., 28000., 30000., 32000., 34000.]))

In [34]:
# CoastFIRE on various numbers
for elem in arr:
    print(elem,PN_cste(elem,i,N))

100000.0 [100000.         107000.         114490.         122504.3
 131079.601      140255.17307    150073.0351849  160578.14764784
 171818.61798319 183845.92124202 196715.13572896 210485.19522998
 225219.15889608 240984.50001881 257853.41502012 275903.15407153]
150000.0 [150000.         160500.         171735.         183756.45
 196619.4015     210382.759605   225109.55277735 240867.22147176
 257727.92697479 275768.88186302 295072.70359343 315727.79284498
 337828.73834412 361476.75002821 386780.12253019 413854.7311073 ]
200000.0 [200000.         214000.         228980.         245008.6
 262159.202      280510.34614    300146.0703698  321156.29529569
 343637.23596638 367691.84248403 393430.27145791 420970.39045997
 450438.31779216 481969.00003762 515706.83004025 551806.30814307]
250000.0 [250000.         267500.         286225.         306260.75
 327699.0025     350637.932675   375182.58796225 401445.36911961
 429546.54495798 459614.80310504 491787.83932239 526212.98807496
 563047.8972

In [42]:
400e3*.04

16000.0